In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import unicodedata
import psutil
import time
import concurrent.futures

In [2]:
psutil.virtual_memory()

svmem(total=17120681984, available=9156161536, percent=46.5, used=7964520448, free=9156161536)

In [3]:
os.cpu_count()

24

In [4]:
# Initializing the card database with empty values 
card_database = pd.DataFrame(columns = [
                                'Card Name', 
                                'Card Type', 
                                'Spell/Trap Property', 
                                'Attribute', 
                                'Types',
                                'Level/Rank', 
                                'ATK', 
                                'DEF', 
                                'LINK',
                                'Pendulum Scale', 
                                'Card Description', 
                                'Card/Attribute/Type Support',
                                'Direct Archetype & Series Support',
                                'Indirect Archetype & Series Support',
                                'Competitive Status (TCG Advanced)',
                                'Reference',
                                ]
                            )
card_database

,Card Name,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference


In [5]:
yugioh_booster_web_page = 'https://yugioh.fandom.com/wiki/Booster_Pack'

cutoff_booster_pack = 'Eternity Code' # Current cutoff that will change over time once new english booster packs come out 
                                      # or new banlist is updated
                                      # Date of update: 26 July 2020

In [6]:
# Function to scrape the specific yugioh web page that contains the urls of all the booster packs up to the current meta
# and returns them in a list format
def get_booster_urls(url, cutoff):
    yugioh_booster_source = requests.get(url)

    if yugioh_booster_source.status_code == 200:
        yugioh_booster_site_html = BeautifulSoup(yugioh_booster_source.text.encode('utf-8'), 'html')

    booster_pack_url_list = list()

    # Appending the urls to a list
    for table in yugioh_booster_site_html.find_all('table', attrs = {'class': "nowraplinks navbox-subgroup"}):
        for ul in table.find_all('ul'):
            for a in ul.find_all('a'):
                booster_pack_url_list.append('https://yugioh.fandom.com' + a['href'])
                if a['title'] == cutoff:
                    break
            if a['title'] == cutoff:
                break
        if a['title'] == cutoff:
            break

    return booster_pack_url_list

In [7]:
booster_pack_url_list = get_booster_urls(yugioh_booster_web_page, cutoff_booster_pack)
print(len(booster_pack_url_list))
booster_pack_url_list

107


['https://yugioh.fandom.com/wiki/Vol.1',
 'https://yugioh.fandom.com/wiki/Vol.2',
 'https://yugioh.fandom.com/wiki/Vol.3',
 'https://yugioh.fandom.com/wiki/Vol.4',
 'https://yugioh.fandom.com/wiki/Vol.5',
 'https://yugioh.fandom.com/wiki/Vol.6',
 'https://yugioh.fandom.com/wiki/Vol.7',
 'https://yugioh.fandom.com/wiki/Booster_1',
 'https://yugioh.fandom.com/wiki/Booster_2',
 'https://yugioh.fandom.com/wiki/Booster_3',
 'https://yugioh.fandom.com/wiki/Booster_4',
 'https://yugioh.fandom.com/wiki/Booster_5',
 'https://yugioh.fandom.com/wiki/Booster_6',
 'https://yugioh.fandom.com/wiki/Booster_7',
 'https://yugioh.fandom.com/wiki/Magic_Ruler_(Japanese)',
 'https://yugioh.fandom.com/wiki/Pharaoh%27s_Servant_(Japanese)',
 'https://yugioh.fandom.com/wiki/Curse_of_Anubis_(set)',
 'https://yugioh.fandom.com/wiki/Thousand_Eyes_Bible',
 'https://yugioh.fandom.com/wiki/Spell_of_Mask',
 'https://yugioh.fandom.com/wiki/Labyrinth_of_Nightmare_(Japanese)',
 'https://yugioh.fandom.com/wiki/Struggle_of

In [8]:
all_other_packs_template = 'https://yugioh.fandom.com/wiki/Template:Packs'
# These are denied card pack URLs that have not been updated with the latest cards from Konami
# or they are typos and have no cards in them
card_pack_denied_urls = ['https://yugioh.fandom.com/wiki/Phantom_Rage_%2B1_Bonus_Pack',
               'https://yugioh.fandom.com/wiki/Rise_of_the_Duelist_%2B1_Bonus_Pack',
               'https://yugioh.fandom.com/wiki/Blazing_Vortex_%2B1_Bonus_Pack',
               'https://yugioh.fandom.com/wiki/Maximum_Gold',
               'https://yugioh.fandom.com/wiki/EX_Value_The_Gold_Box_%2B_ABYR%26CBLZ',
               'https://yugioh.fandom.com/wiki/EX_Value_The_Gold_Box_%2B_GS2013',
               'https://yugioh.fandom.com/wiki/Legendary_Duelists:_Rage_of_Ra',
               'https://yugioh.fandom.com/wiki/World_Premiere_Pack_2020',
               'https://yugioh.fandom.com/wiki/Dragons_of_Legend:_The_Complete_Series',
               'https://yugioh.fandom.com/wiki/Deck_Build_Pack:_Genesis_Impactors']

decks_template = 'https://yugioh.fandom.com/wiki/Template:Decks'
# Denied URLs for the deck URLs
deck_denied_urls = ['https://yugioh.fandom.com/wiki/Yu-Gi-Oh!_Rush_Duel_Starter_Deck:_Yuga_-_Open!_Sevens_Road!!',
                    'https://yugioh.fandom.com/wiki/Yu-Gi-Oh!_Rush_Duel_Starter_Deck:_Luke_-_Explosive_Conquest!_Dragears!!',
                    'https://yugioh.fandom.com/wiki/Yu-Gi-Oh!_Rush_Duel_Starter_Deck_Set_-_Yuga_vs._Luke',
                    'https://yugioh.fandom.com/wiki/Structure_Deck_R:_Dragunity_Drive']

reprint_template = 'https://yugioh.fandom.com/wiki/Template:Reprint_sets'
reprint_denied_urls = []

collectible_tins_template = 'https://yugioh.fandom.com/wiki/Template:Collectible_tins'
collectible_tins_denied_urls = ['https://yugioh.fandom.com/wiki/Tin_of_Lost_Memories', 
                                'https://yugioh.fandom.com/wiki/Duelist_Pack_Collection_Tin_2009']

In [9]:
# Function is used specifically for yugioh sets that are not booster packs like duelist packs, event packs
# as well as yugioh decks like structure and starter decks, reprint sets, and collectible tins 
# to get the urls of all these other sets from their Original Template web page 
def get_other_sets_url(url, denied_urls):
    template_source = requests.get(url)
    
    if template_source.status_code == 200:
        template_html = BeautifulSoup(template_source.text.encode('utf-8'), 'html.parser')
        
    set_url_list = list()
    for li in template_html.find('table', attrs = {'class': "navbox"}).find_all('li'):
        for a in li.find_all('a'):
            try:
                set_url_list.append('https://yugioh.fandom.com' + a['href'])
            except KeyError:
                pass
    set_url_list.pop(0) # In a template URL format, the first link is useless, and thus is popped        
    set_url_list = list(set(set_url_list))
    
    set_url_list = [link for link in set_url_list if link not in denied_urls]
    
    return set_url_list

In [10]:
other_card_packs_url_list = get_other_sets_url(all_other_packs_template, card_pack_denied_urls)
other_card_packs_url_list

['https://yugioh.fandom.com/wiki/Surpassing_10000_Cards_Commemoration_Special_Pack',
 'https://yugioh.fandom.com/wiki/Special_Pack_Vol.3',
 'https://yugioh.fandom.com/wiki/World_Superstars',
 'https://yugioh.fandom.com/wiki/Dark_Ceremony_Edition',
 'https://yugioh.fandom.com/wiki/Template:Collectible_tins',
 'https://yugioh.fandom.com/wiki/Extra_Pack_2016',
 'https://yugioh.fandom.com/wiki/Premium_Pack_12',
 'https://yugioh.fandom.com/wiki/Special_Pack_20th_Anniversary_Edition_Vol.5',
 'https://yugioh.fandom.com/wiki/Hidden_Arsenal_3',
 'https://yugioh.fandom.com/wiki/Template:Tournament_Packs',
 'https://yugioh.fandom.com/wiki/Legendary_Duelists:_Magical_Hero',
 'https://yugioh.fandom.com/wiki/Secret_Slayers',
 'https://yugioh.fandom.com/wiki/Exclusive_Pack',
 'https://yugioh.fandom.com/wiki/Deck_Build_Pack:_Spirit_Warriors',
 'https://yugioh.fandom.com/wiki/Duelist_Pack:_Legend_Duelist_6',
 'https://yugioh.fandom.com/wiki/Jump_Festa_2012_-_Promotion_Pack_A',
 'https://yugioh.fandom.c

In [11]:
decks_url_list = get_other_sets_url(decks_template, deck_denied_urls)
decks_url_list

['https://yugioh.fandom.com/wiki/Structure_Deck:_Yugi_Volume_2',
 'https://yugioh.fandom.com/wiki/Starter_Deck_2006',
 'https://yugioh.fandom.com/wiki/Realm_of_Light_Structure_Deck',
 'https://yugioh.fandom.com/wiki/Duel_Starter_Deck',
 'https://yugioh.fandom.com/wiki/Zombie_World_Structure_Deck',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Lost_Sanctuary',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Master_of_Pendulum',
 'https://yugioh.fandom.com/wiki/Millennium_Box_Gold_Edition',
 'https://yugioh.fandom.com/wiki/Starter_Deck_2010',
 'https://yugioh.fandom.com/wiki/Structure_Deck_R:_Undead_World',
 'https://yugioh.fandom.com/wiki/Template:Collectible_tins',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Marik_(OCG)',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Warrior%27s_Strike',
 'https://yugioh.fandom.com/wiki/Rise_of_the_Dragon_Lords_Structure_Deck',
 'https://yugioh.fandom.com/wiki/Starter_Deck:_Yugi_Evolution',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_P

In [12]:
reprints_url_list = get_other_sets_url(reprint_template, reprint_denied_urls)
reprints_url_list

['https://yugioh.fandom.com/wiki/Precious_Pack_Volume_1',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_4',
 'https://yugioh.fandom.com/wiki/Template:Shonen_Jump_promotional_cards',
 'https://yugioh.fandom.com/wiki/Booster_R3',
 'https://yugioh.fandom.com/wiki/Dark_Beginning_1',
 'https://yugioh.fandom.com/wiki/Template:Promotional_cards',
 'https://yugioh.fandom.com/wiki/Revival_of_Black_Demons_Dragon',
 'https://yugioh.fandom.com/wiki/Expert_Edition_Volume_1',
 'https://yugioh.fandom.com/wiki/Template:Video_game_promotional_cards',
 'https://yugioh.fandom.com/wiki/Dark_Revelation_Volume_1',
 'https://yugioh.fandom.com/wiki/Retro_Pack_2',
 'https://yugioh.fandom.com/wiki/Template:V_Jump_promotional_cards',
 'https://yugioh.fandom.com/wiki/Template:Collectible_tins',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_3',
 'https://yugioh.fandom.com/wiki/Template:Packs',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_2',
 'https://yugioh.fandom.com/wiki/Template:

In [13]:
collectible_tins_url_list = get_other_sets_url(collectible_tins_template, collectible_tins_denied_urls)
collectible_tins_url_list

['https://yugioh.fandom.com/wiki/Duelist_Pack_Collection',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2007',
 'https://yugioh.fandom.com/wiki/Booster_Pack_Collectors_Tin_2006',
 'https://yugioh.fandom.com/wiki/Template:Shonen_Jump_promotional_cards',
 'https://yugioh.fandom.com/wiki/Template:Promotional_cards',
 'https://yugioh.fandom.com/wiki/Template:Video_game_promotional_cards',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_Exclusive_2009',
 'https://yugioh.fandom.com/wiki/2013_Collectible_Tins_Wave_1',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2009_Wave_2',
 'https://yugioh.fandom.com/wiki/Premium_Collection_Tin',
 'https://yugioh.fandom.com/wiki/2018_Mega-Tins',
 'https://yugioh.fandom.com/wiki/Duelist_Pack_Collection_Tin_2008',
 'https://yugioh.fandom.com/wiki/Template:V_Jump_promotional_cards',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2006_Wave_2',
 'https://yugioh.fandom.com/wiki/Template:Packs',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_200

In [14]:
# Function to filter the true card pack and deck URLs from the URLs that have no table of cards in them
# or just random hyperlinks in general
def url_filter(url, index):
    try:
        pd.read_html(url, attrs = {'class': 'wikitable'})
    except ValueError: 
        return index

# Function used to pop the indexes of the set urls that did not pass the url_filter function
def popping_indexes(popped_indexes, url_list):
    popped_indexes = [i for i in popped_indexes if i != None]
    
    for index in sorted(popped_indexes, reverse = True):
        del url_list[index]
    
    return url_list

In [15]:
time1 = time.perf_counter()

if __name__ == '__main__':
    # Multi-Threading to speed up the webscraping and filtering the actual card pack URLs
    with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
        popped_indexes_iterable = executor.map(url_filter, other_card_packs_url_list, range(len(other_card_packs_url_list)))
    print(f'{time.perf_counter() - time1} seconds')
    
    other_card_packs_url_list = popping_indexes(list(popped_indexes_iterable), other_card_packs_url_list)

other_card_packs_url_list

169.656347 seconds


['https://yugioh.fandom.com/wiki/Surpassing_10000_Cards_Commemoration_Special_Pack',
 'https://yugioh.fandom.com/wiki/Special_Pack_Vol.3',
 'https://yugioh.fandom.com/wiki/World_Superstars',
 'https://yugioh.fandom.com/wiki/Dark_Ceremony_Edition',
 'https://yugioh.fandom.com/wiki/Extra_Pack_2016',
 'https://yugioh.fandom.com/wiki/Premium_Pack_12',
 'https://yugioh.fandom.com/wiki/Special_Pack_20th_Anniversary_Edition_Vol.5',
 'https://yugioh.fandom.com/wiki/Hidden_Arsenal_3',
 'https://yugioh.fandom.com/wiki/Legendary_Duelists:_Magical_Hero',
 'https://yugioh.fandom.com/wiki/Secret_Slayers',
 'https://yugioh.fandom.com/wiki/Exclusive_Pack',
 'https://yugioh.fandom.com/wiki/Deck_Build_Pack:_Spirit_Warriors',
 'https://yugioh.fandom.com/wiki/Duelist_Pack:_Legend_Duelist_6',
 'https://yugioh.fandom.com/wiki/Jump_Festa_2012_-_Promotion_Pack_A',
 'https://yugioh.fandom.com/wiki/Hidden_Arsenal_5:_Steelswarm_Invasion',
 'https://yugioh.fandom.com/wiki/Collectors_Pack_2017',
 'https://yugioh.f

In [16]:
time1 = time.perf_counter()

if __name__ == '__main__':
    # Multi-Threading to speed up the webscraping and filtering the actual deck URLs
    with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
        popped_indexes_iterable = executor.map(url_filter, decks_url_list, range(len(decks_url_list)))
    print(f'{time.perf_counter() - time1} seconds')
    
    decks_url_list = popping_indexes(list(popped_indexes_iterable), decks_url_list)

    decks_url_list.extend(['https://yugioh.fandom.com/wiki/Set_Card_Lists:Structure_Deck:_Lord_of_the_Storm_(TCG-EN)',
                           'https://yugioh.fandom.com/wiki/Set_Card_Lists:Structure_Deck:_Dinosaur%27s_Rage_(TCG-EN)',
                           'https://yugioh.fandom.com/wiki/Set_Card_Lists:Structure_Deck:_Invincible_Fortress_(TCG-EN)',
                           'https://yugioh.fandom.com/wiki/Set_Card_Lists:Structure_Deck:_Spellcaster%27s_Judgment_(TCG-EN)'])

decks_url_list

97.11844969999999 seconds


['https://yugioh.fandom.com/wiki/Structure_Deck:_Yugi_Volume_2',
 'https://yugioh.fandom.com/wiki/Starter_Deck_2006',
 'https://yugioh.fandom.com/wiki/Realm_of_Light_Structure_Deck',
 'https://yugioh.fandom.com/wiki/Duel_Starter_Deck',
 'https://yugioh.fandom.com/wiki/Zombie_World_Structure_Deck',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Lost_Sanctuary',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Master_of_Pendulum',
 'https://yugioh.fandom.com/wiki/Millennium_Box_Gold_Edition',
 'https://yugioh.fandom.com/wiki/Starter_Deck_2010',
 'https://yugioh.fandom.com/wiki/Structure_Deck_R:_Undead_World',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Marik_(OCG)',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Warrior%27s_Strike',
 'https://yugioh.fandom.com/wiki/Rise_of_the_Dragon_Lords_Structure_Deck',
 'https://yugioh.fandom.com/wiki/Starter_Deck:_Yugi_Evolution',
 'https://yugioh.fandom.com/wiki/Structure_Deck:_Pendulum_Domination',
 'https://yugioh.fandom.com/wiki/Starter

In [17]:
time1 = time.perf_counter()

if __name__ == '__main__':
    # Multi-Threading to speed up the webscraping and filtering the actual reprint URLs
    with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
        popped_indexes_iterable = executor.map(url_filter, reprints_url_list, range(len(reprints_url_list)))
    print(f'{time.perf_counter() - time1} seconds')
    
    reprints_url_list = popping_indexes(list(popped_indexes_iterable), reprints_url_list)

reprints_url_list

28.5137603 seconds


['https://yugioh.fandom.com/wiki/Precious_Pack_Volume_1',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_4',
 'https://yugioh.fandom.com/wiki/Booster_R3',
 'https://yugioh.fandom.com/wiki/Dark_Beginning_1',
 'https://yugioh.fandom.com/wiki/Revival_of_Black_Demons_Dragon',
 'https://yugioh.fandom.com/wiki/Expert_Edition_Volume_1',
 'https://yugioh.fandom.com/wiki/Dark_Revelation_Volume_1',
 'https://yugioh.fandom.com/wiki/Retro_Pack_2',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_3',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_2',
 'https://yugioh.fandom.com/wiki/Dark_Legends',
 'https://yugioh.fandom.com/wiki/Duelist_Legacy_Volume.2',
 'https://yugioh.fandom.com/wiki/Phantom_God',
 'https://yugioh.fandom.com/wiki/Expert_Edition_Volume_4',
 'https://yugioh.fandom.com/wiki/Duelist_Edition_Volume_1',
 'https://yugioh.fandom.com/wiki/Metal_Raiders_(Japanese)',
 'https://yugioh.fandom.com/wiki/Duelist_Legacy_Volume.1',
 'https://yugioh.fandom.com/wiki/Dark_R

In [18]:
time1 = time.perf_counter()

if __name__ == '__main__':
    # Multi-Threading to speed up the webscraping and filtering the actual collectible tins URLs
    with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
        popped_indexes_iterable = executor.map(url_filter, collectible_tins_url_list, range(len(collectible_tins_url_list)))
    print(f'{time.perf_counter() - time1} seconds')
    
    collectible_tins_url_list = popping_indexes(list(popped_indexes_iterable), collectible_tins_url_list)

collectible_tins_url_list

19.250217499999962 seconds


['https://yugioh.fandom.com/wiki/Duelist_Pack_Collection',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2007',
 'https://yugioh.fandom.com/wiki/Booster_Pack_Collectors_Tin_2006',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_Exclusive_2009',
 'https://yugioh.fandom.com/wiki/2013_Collectible_Tins_Wave_1',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2009_Wave_2',
 'https://yugioh.fandom.com/wiki/Premium_Collection_Tin',
 'https://yugioh.fandom.com/wiki/2018_Mega-Tins',
 'https://yugioh.fandom.com/wiki/Duelist_Pack_Collection_Tin_2008',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2006_Wave_2',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2008_Wave_2',
 'https://yugioh.fandom.com/wiki/Collectible_Tins_2010',
 'https://yugioh.fandom.com/wiki/2015_Mega-Tins',
 'https://yugioh.fandom.com/wiki/Booster_Pack_Collectors_Tin_2003',
 'https://yugioh.fandom.com/wiki/Booster_Pack_Collectors_Tin_2007',
 'https://yugioh.fandom.com/wiki/Duelist_Pack_Collection_Tin_2011',
 'http

In [19]:
all_url_list = booster_pack_url_list + other_card_packs_url_list + decks_url_list + reprints_url_list + collectible_tins_url_list
len(all_url_list)

638

In [20]:
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Vol.1'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Vol.2'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Booster_1'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Pharaoh%27s_Servant_(Japanese)'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Spell_Ruler'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Judgment_of_the_Light'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Rising_Rampage'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Legend_of_Blue_Eyes_White_Dragon'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Advent_of_Union'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Order_of_Chaos'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Secrets_of_Eternity'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Magic_Ruler_(Japanese)'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Threat_of_the_Dark_Demon_World'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Legendary_Duelists:_Season_1'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Storm_of_Ragnarok'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/Rise_of_the_Duelist'
#booster_pack_url = 'https://yugioh.fandom.com/wiki/World_Championship_2010_Card_Pack'

In [21]:
#deck = 'https://yugioh.fandom.com/wiki/Starter_Deck_Yugi_Reloaded'
#deck = 'https://yugioh.fandom.com/wiki/Starter_Deck_2019'
#deck = 'https://yugioh.fandom.com/wiki/Speed_Duel_Starter_Decks:_Twisted_Nightmares'
#deck = 'https://yugioh.fandom.com/wiki/Speed_Duel_Starter_Decks:_Match_of_the_Millennium'
#deck = 'https://yugioh.fandom.com/wiki/Speed_Duel_Starter_Decks:_Ultimate_Predators'
#deck = 'https://yugioh.fandom.com/wiki/Structure_Deck:_Marik_(OCG)'
#deck = 'https://yugioh.fandom.com/wiki/Structure_Deck_-_Machine_Re-Volt'
#deck = 'https://yugioh.fandom.com/wiki/Structure_Deck_-_Dinosaur%27s_Rage'
#deck = 'https://yugioh.fandom.com/wiki/Set_Card_Lists:Structure_Deck:_Lord_of_the_Storm_(TCG-EN)'
#deck = 'https://yugioh.fandom.com/wiki/Structure_Deck:_Masters_of_the_Spiritual_Arts'
#deck = 'https://yugioh.fandom.com/wiki/Structure_Deck_R:_Dragunity_Drive'
#deck = 'https://yugioh.fandom.com/wiki/Duelist_Box'
#deck = 'https://yugioh.fandom.com/wiki/Legendary_Hero_Decks'
#deck = 'https://yugioh.fandom.com/wiki/Duel_Devastator'

In [22]:
# Function that scrapes the card set URL (packs, decks, reprint sets, tins) and return all the urls of 
# each card from the card set in a list format
def get_card_urls(card_set_url):
    card_set_source = requests.get(card_set_url)

    if card_set_source.status_code == 200:
        card_set_html = BeautifulSoup(card_set_source.text.encode('utf-8'), 'html.parser')

    class_or_id = 'class'
    attribute_name = "wikitable"
    # Code hacks for 2 card pack URLs
    if card_set_url == 'https://yugioh.fandom.com/wiki/Duelist_Pack:_Kite':
        attribute_name = "sortable"
    elif card_set_url == 'https://yugioh.fandom.com/wiki/Collection_Pack_2020':
        class_or_id = 'id'
        attribute_name = 'Top_table'
        
    card_url_list = list()
    
    for card_table in card_set_html.find_all('table', attrs = {class_or_id: attribute_name}):
        # Building a table of hyperlinks because pd.read_html does not read the hyperlinks, but only reads the unlinked
        # text of tables
        record = list()
        for tr in card_table.findAll("tr"):
            ths = tr.findAll("th")
            if ths != []:
                columns = [th.text.replace('\n', '').strip() for th in ths]
            else:
                row = list()
                for td in tr.find_all('td'):
                    try:
                        row.append(td.a['href'])
                    except KeyError:
                        row.append(td.a.text)
                    except TypeError:
                        row.append(td.text)
                record.append(row)

        card_set_df = pd.DataFrame(data = record, columns = columns)
        try:
            card_set_df.rename(columns = {'English name': 'Card Name'}, inplace = True)
            # Raise keyword is used because KeyError was handled previously above, thus raise is
            # needed to  reraise the KeyError if it comes up again
            raise KeyError() 
        except KeyError:
            card_set_df.rename(columns = {'Name': 'Card Name'}, inplace = True)
        finally:
            # Filtering out the links with no 'wiki' in them, all legit cards in the yugioh fandom site has wiki in their urls
            card_set_df['Card Name'] = card_set_df['Card Name'].apply(lambda x: np.where(x.find('wiki') != -1, x, 'Not a link'))
            card_set_df = card_set_df[card_set_df['Card Name'] != 'Not a link']    
            # Adding yugioh.fandom in front of the link to complete the URL
            card_set_df['Card Name'] = card_set_df['Card Name'].apply(lambda x: 'https://yugioh.fandom.com' + x) 
            card_url_list.extend(list(card_set_df['Card Name']))
            
    card_url_list = list(set(card_url_list))
    return card_url_list

In [23]:
#get_card_urls(booster_pack_url)
#get_card_urls(deck)

In [24]:
time1 = time.perf_counter()

if __name__ == '__main__':
    # Multi-threading is used here because webscraping is an I/O bound activity, and specify max_workers
    # argument because the default value will not render the multi-threading successful
    with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
        card_url_iterable = executor.map(get_card_urls, all_url_list)
    print(f'{time.perf_counter() - time1} seconds')

# Flattening the iterable because the iterable is a list of lists, each card pack has its own list of cards
# so to combine all card urls from different card packs, we need to flatten the list
card_url_list = [item for sublist in list(card_url_iterable) for item in sublist]

# To remove duplicate URLs, since the same card can exist in multiple different packs
card_url_list = list(set(card_url_list))
card_url_list

136.78362119999997 seconds


['https://yugioh.fandom.com/wiki/Divine_Grace_-_Northwemko',
 'https://yugioh.fandom.com/wiki/Double_Summon',
 'https://yugioh.fandom.com/wiki/Zoodiac_Gathering',
 'https://yugioh.fandom.com/wiki/The_Legendary_Exodia_Incarnate',
 'https://yugioh.fandom.com/wiki/Dark_Designator',
 'https://yugioh.fandom.com/wiki/Malefic_Tune',
 'https://yugioh.fandom.com/wiki/Hero_Flash!!',
 'https://yugioh.fandom.com/wiki/Zoma_the_Spirit',
 'https://yugioh.fandom.com/wiki/Lilith,_Lady_of_Lament',
 'https://yugioh.fandom.com/wiki/Favorite_Hero',
 'https://yugioh.fandom.com/wiki/Deck_Lockdown',
 'https://yugioh.fandom.com/wiki/Apocatequil',
 'https://yugioh.fandom.com/wiki/Genex_Turbine',
 'https://yugioh.fandom.com/wiki/Ikelos,_the_Dream_Mirror_Mara',
 'https://yugioh.fandom.com/wiki/Labyrinth_Tank',
 'https://yugioh.fandom.com/wiki/Phantom_of_Chaos',
 'https://yugioh.fandom.com/wiki/Revival_Golem',
 'https://yugioh.fandom.com/wiki/Rush_Recklessly',
 'https://yugioh.fandom.com/wiki/Toon_Dark_Magician',


In [25]:
len(card_url_list)

10570

In [26]:
card_url_list.index('https://yugioh.fandom.com/wiki/Summon_Sorceress')

ValueError: 'https://yugioh.fandom.com/wiki/Summon_Sorceress' is not in list

In [27]:
card_url_list.extend(['https://yugioh.fandom.com/wiki/Summon_Sorceress'])
len(list(set(card_url_list)))

10571

In [28]:
#url = 'https://yugioh.fandom.com/wiki/Elemental_HERO_Flame_Wingman'
#url = 'https://yugioh.fandom.com/wiki/Firewall_Dragon'
#url = 'https://yugioh.fandom.com/wiki/Hitotsu-Me_Giant'
#url = 'https://yugioh.fandom.com/wiki/Dark_Magician'
#url = 'https://yugioh.fandom.com/wiki/Seven_Tools_of_the_Bandit'
#url = 'https://yugioh.fandom.com/wiki/Slifer_the_Sky_Dragon'
#url = 'https://yugioh.fandom.com/wiki/Galaxy_Soldier'
#url = 'https://yugioh.fandom.com/wiki/Swords_of_Revealing_Light'
#url = 'https://yugioh.fandom.com/wiki/D/D/D_Super_Doom_King_Purple_Armageddon'
#url = 'https://yugioh.fandom.com/wiki/Blackwing_-_Gofu_the_Vague_Shadow'
#url = 'https://yugioh.fandom.com/wiki/Raigeki'
#url = 'https://yugioh.fandom.com/wiki/Astrograph_Sorcerer'
#url = 'https://yugioh.fandom.com/wiki/Fiend%27s_Hand'
#url = 'https://yugioh.fandom.com/wiki/High_Tide_Gyojin'
#url = 'https://yugioh.fandom.com/wiki/Forbidden_Dark_Contract_with_the_Swamp_King'
#url = 'https://yugioh.fandom.com/wiki/Curious,_the_Lightsworn_Dominion'
#url = 'https://yugioh.fandom.com/wiki/Topologic_Bomber_Dragon'
#url = 'https://yugioh.fandom.com/wiki/Catch_of_the_Day'
#url = 'https://yugioh.fandom.com/wiki/Mythic_Depths'
#url = 'https://yugioh.fandom.com/wiki/Maxx_%22C%22'
#url = 'https://yugioh.fandom.com/wiki/Danger!_Nessie!'
#url = 'https://yugioh.fandom.com/wiki/Altergeist_Multifaker'
#url = 'https://yugioh.fandom.com/wiki/Grinder_Golem'
#url = 'https://yugioh.fandom.com/wiki/Garmr_of_the_Nordic_Beasts'
#url = 'https://yugioh.fandom.com/wiki/ABC-Dragon_Buster'
#url = 'https://yugioh.fandom.comInner Conflict'
#url = 'https://yugioh.fandom.com/wiki/Dinosaur_Kingdom'
#url = 'https://yugioh.fandom.com/wiki/Simorgh,_Bird_of_Divinity'

In [29]:
# Function that scrapes the card url in its argument and returns the details of the card in a user-readable
# format in the form of a dictionary
def get_card_details(url):
    # Try-block code to read the table in the card url, and if the url passed is not a card URL,
    # the except-block will run and print out that url that is faulty
    try:
        card_details_df = pd.read_html(url, attrs = {'class': "cardtable"})[0]
    except ValueError:
        print(url)
        return None
    else:
        card_details_df.drop(columns = [card_details_df.columns[0]], inplace = True)
        card_details_df.rename(columns = {card_details_df.columns[1]: 'Card Properties'}, inplace = True)
        card_details_df.set_index(card_details_df.columns[0], inplace = True)

        # Variables of the Yugioh cards I am interested in 
        card_name = 'N/A'
        card_type = 'N/A'
        spell_trap_property = 'N/A'
        attribute = 'N/A'
        types = 'N/A'
        level_rank = 'N/A'
        ATK = 'N/A'
        DEF = 'N/A'
        LINK = 'N/A'
        pendulum_scale = 'N/A'
        card_description = 'N/A'
        card_attribute_type_support = set()
        direct_archetype_series_support = set()
        indirect_archetype_series_support = set()
        competitive_status = 'N/A'

        # Function to handle the ATK and DEF/LINK properties of a monster card
        def atk_def_link_parser(atk_def_link):
            pattern_atk = r'^\w*\?*'
            pattern_def_link = r'\w*\?*$'

            # re.match only checks for a match at the beginning of the string, but it is quicker than search
            try:
                ATK = int(re.match(pattern_atk, atk_def_link).group(0))
            except (ValueError):
                ATK = re.match(pattern_atk, atk_def_link).group(0)

            # re.search checks for a match anywhere in the string
            try:
                DEF_LINK = int(re.search(pattern_def_link, atk_def_link).group(0))
            except ValueError:
                DEF_LINK = re.search(pattern_def_link, atk_def_link).group(0)

            return (ATK, DEF_LINK)

        # Try block to handle the card information, and any errors that occur
        # Some cards may have careless mistakes that or missing information that is not in the norm and impossible
        # to check the specific error encountered
        try:
            # Block of code to handle Card Names
            # Some cards have multiple English indexes, and so their names are a series of anything with
            # the English index, and this if else block of code helps to separate the true english 
            # name from all other values in that index
            if isinstance(card_details_df['Card Properties'].loc['English'], str):
                card_name = card_details_df['Card Properties'].loc['English']
            else:
                card_name = card_details_df['Card Properties'].loc['English'][0]

            # Block of code to handle Card Types (Monster, Spell, Trap)
            card_type = card_details_df['Card Properties'].loc['Card type']

            if card_details_df['Card Properties'].loc['Card type'] != 'Monster': # Block of code to handle spell and trap cards
                spell_trap_property = card_details_df['Card Properties'].loc['Property']
            else: # Block of code to handle monster cards
                attribute = card_details_df['Card Properties'].loc['Attribute']

                try:
                    types = card_details_df['Card Properties'].loc['Type'] + ' / Normal'
                except KeyError:
                    types = card_details_df['Card Properties'].loc['Types']

                # Try-Block to handle level/rank discrepancies of Xyz and non-Xyz Monsters
                try:
                    level_rank = int(card_details_df['Card Properties'].loc['Level'])
                except KeyError:
                    try:
                        level_rank = int(card_details_df['Card Properties'].loc['Rank'])
                    except KeyError: # For Link Monsters with no level or rank
                        level_rank = 'N/A'

                # Try-Block to handle ATK, DEF, and LINK discrepancies of LINK and non-LINK monsters
                try:
                    atk_def_link = card_details_df['Card Properties'].loc['ATK / DEF']
                    DEF = atk_def_link_parser(atk_def_link)[1]
                    LINK = 'N/A'
                except KeyError:
                    atk_def_link = card_details_df['Card Properties'].loc['ATK / LINK']
                    LINK = atk_def_link_parser(atk_def_link)[1]
                    DEF = 'N/A'
                finally:
                    ATK = atk_def_link_parser(atk_def_link)[0]

                # Try-Block to handle the properties of Pendulum Monsters
                try:
                    pendulum_scale = int(card_details_df['Card Properties'].loc['Pendulum Scale'])
                except KeyError:
                    pendulum_scale = 'N/A'

            # Block of code to handle the card description of all cards and make them readable
            source = requests.get(url)
            if source.status_code == 200:
                site_html = BeautifulSoup(source.text.encode('utf-8'), 'html.parser')
            uncleaned_description = unicodedata.normalize("NFKD", site_html.find_all('td', attrs = {'class': "navbox-list"})[0].text.replace('\n', ''))
            card_description = re.sub(r'(?<=[.,])(?=[^\s])', r' ', uncleaned_description) # Pendulum Monsters text have this issue
            card_description = re.sub(r'(?<=[a-z])(?=[A-Z])', '. ',  card_description) # Link and Synchro Monsters have this issue 
            card_description = re.sub(r'(?<=[a-z]["])(?=[A-Z])', r'. ', card_description) # Fusion monsters text have this issue        

            # Block of code to handle which archetype/series/attribute/type/individual cards that each card supports
            if site_html.find('div', attrs = {'class': "hlist"}).dt:
                for div in site_html.find_all('div', attrs = {'class': "hlist"}):
                    try:
                        if div.dt.text.replace('\n', '').strip() == 'Supports':
                            for dd in div.find_all('dd'):
                                card_attribute_type_support.add(dd.text.replace('\n', '').strip())
                        elif div.dt.text.replace('\n', '').strip() == 'Archetypes and series' or div.dt.text.replace('\n', '').strip() == 'Supports archetypes':
                            for dd in div.find_all('dd'):
                                direct_archetype_series_support.add(dd.text.replace('\n', '').strip())
                        elif div.dt.text.replace('\n', '').strip() == 'Related to archetypes and series':
                            for dd in div.find_all('dd'):
                                indirect_archetype_series_support.add(dd.text.replace('\n', '').strip())
                        else:
                            break
                    except AttributeError:
                        break
            else:
                pass

            # Block of code to handle the competitive status of each card in the current TCG Advanced meta
            pattern_to_search = 'TCG Advanced'
            pattern_to_extract = r'^\w+'
            if isinstance(card_details_df['Card Properties'].loc['Statuses'], str):
                competitive_status = card_details_df['Card Properties'].loc['Statuses']
                if re.search(pattern_to_search, competitive_status):
                    competitive_status = re.match(pattern_to_extract, competitive_status).group()
                else:
                    pass
            else:
                for status in card_details_df['Card Properties'].loc['Statuses'].values:
                    if re.search(pattern_to_search, status):
                        competitive_status = re.match(pattern_to_extract, status).group()

        except Exception as e:
            card_type = e

        # Putting all the card informations in a dictionary format
        card_dict = {
            'Card Name': card_name,
            'Card Type': card_type,
            'Spell/Trap Property': spell_trap_property,
            'Attribute': attribute,
            'Types': types,
            'Level/Rank': str(level_rank),
            'ATK': str(ATK),
            'DEF': str(DEF),
            'LINK': str(LINK),
            'Pendulum Scale': str(pendulum_scale),
            'Card Description': card_description,
            'Card/Attribute/Type Support': card_attribute_type_support,
            'Direct Archetype & Series Support': direct_archetype_series_support,
            'Indirect Archetype & Series Support': indirect_archetype_series_support,
            'Competitive Status (TCG Advanced)': competitive_status,
            'Reference': url,
            }

        return card_dict

In [30]:
# card_details_dict = get_card_details(url)
# card_details_dict

In [33]:
# For some reason, Ancient Gear is added as a URL list to the card_url, so I pop it out here, because 
# the link goes to an archetype link and not a card link
card_url_list.index('https://yugioh.fandom.com/wiki/Ancient_Gear')

189

In [34]:
card_url_list.pop(189)

'https://yugioh.fandom.com/wiki/Ancient_Gear'

In [35]:
len(card_url_list)

10570

In [36]:
time1 = time.perf_counter()

if __name__ == '__main__':
    # Multi-threading is used here because web scraping is an I/O bound activity, and specify max_workers
    # argument because the default value will not render the multi-threading successful
    with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
        card_dict_iterable = executor.map(get_card_details, card_url_list)
    print(f'{time.perf_counter() - time1} seconds')
    
card_dict_list = list(card_dict_iterable)
card_dict_list

1756.0144305 seconds


[{'Card Name': 'Divine Grace - Northwemko',
  'Card Type': 'Monster',
  'Spell/Trap Property': 'N/A',
  'Attribute': 'LIGHT',
  'Types': 'Spellcaster / Ritual / Effect',
  'Level/Rank': '7',
  'ATK': '2700',
  'DEF': '1200',
  'LINK': 'N/A',
  'Pendulum Scale': 'N/A',
  'Card Description': 'This card can only be Ritual Summoned with the Ritual Spell Card, "Ritual of Grace". When this card is Ritual Summoned, select a number of face-up cards on the field (except this card), up to the number of monsters you used for the Ritual Summon. While any of the selected cards are face-up on the field, this card cannot be destroyed by card effects.',
  'Card/Attribute/Type Support': set(),
  'Direct Archetype & Series Support': set(),
  'Indirect Archetype & Series Support': {'Djinn of Rituals'},
  'Competitive Status (TCG Advanced)': 'Unlimited',
  'Reference': 'https://yugioh.fandom.com/wiki/Divine_Grace_-_Northwemko'},
 {'Card Name': 'Double Summon',
  'Card Type': 'Spell',
  'Spell/Trap Propert

In [37]:
len(card_dict_list)

10570

In [38]:
card_database = pd.DataFrame(card_dict_list)
# Sets are not hashable so it needs to be converted to a string before checking for duplicates,
# and then getting the index of the kept rows and locating it from the original card database dataframe
# so that the sets are kept intact
card_database = card_database.iloc[card_database.astype('str').drop_duplicates(subset = 'Card Name', keep = 'first').index].set_index('Card Name')
card_database

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Divine Grace - Northwemko,Monster,N/A,LIGHT,Spellcaster / Ritual / Effect,7,2700,1200,N/A,N/A,This card can only be Ritual Summoned with the...,{},{},{Djinn of Rituals},Unlimited,https://yugioh.fandom.com/wiki/Divine_Grace_-_...
Double Summon,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,You can conduct 2 Normal Summons/Sets this tur...,{},{},{},Unlimited,https://yugioh.fandom.com/wiki/Double_Summon
Zoodiac Gathering,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"If a ""Zoodiac"" Xyz Monster you control would d...",{Xyz Monster},{Zoodiac},{},Unlimited,https://yugioh.fandom.com/wiki/Zoodiac_Gathering
The Legendary Exodia Incarnate,Monster,N/A,DARK,Spellcaster / Effect,10,?,0,N/A,N/A,Cannot be Normal Summoned/Set. Must be Special...,{},"{Forbidden One, Exodia}",{},Unlimited,https://yugioh.fandom.com/wiki/The_Legendary_E...
Dark Designator,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Declare 1 Monster Card name. If the declared c...,{},{Designator},{},Unlimited,https://yugioh.fandom.com/wiki/Dark_Designator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iron Hans,Monster,N/A,EARTH,Warrior / Effect,4,1200,800,N/A,N/A,If this card is Summoned: You can Special Summ...,"{Iron Knight, Golden Castle of Stromberg}",{Fairy Tale},{},Unlimited,https://yugioh.fandom.com/wiki/Iron_Hans
Frightfur March,Trap,Counter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"When an opponent's Spell/Trap Card, or monster...",{Fusion Monster},{Frightfur},{},Unlimited,https://yugioh.fandom.com/wiki/Frightfur_March
Shapesister,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Special Summon this card as a Normal Monster (...,{},{Trap Monster},{},Unlimited,https://yugioh.fandom.com/wiki/Shapesister


In [39]:
# Checking to see how many of the erroneous card were added
card_database[(card_database['Card Type'] != 'Monster') & (card_database['Card Type'] != 'Spell') & (card_database['Card Type'] != 'Trap')]

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Token,'Attribute',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/Token
Millennium Necklace,'Card type',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/Millennium_Neck...
Valley Token,'Attribute',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/Valley_Token
It's a Toon World!,'Card type',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/It%27s_a_Toon_W...
Millennium Eye,'Card type',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/Millennium_Eye_...
Set Sail for The Kingdom,'Card type',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/Set_Sail_for_Th...
Ma'at,'Statuses',N/A,LIGHT,Fairy / Effect,10,?,?,N/A,N/A,Cannot be Normal Summoned/Set. Must be Special...,"{Dragon, Fairy, LIGHT}",{},"{Darkness, Kuriboh}",N/A,https://yugioh.fandom.com/wiki/Ma%27at
The World’s Greatest Fisherman,'Card type',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/The_World%E2%80...
Pal-O'Mine-zation!,'Card type',N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,{},{},{},N/A,https://yugioh.fandom.com/wiki/Pal-O%27Mine-za...


In [40]:
card_database.loc['Call of the Haunted']

Card Type                                                                           Trap
Spell/Trap Property                                                           Continuous
Attribute                                                                            N/A
Types                                                                                N/A
Level/Rank                                                                           N/A
ATK                                                                                  N/A
DEF                                                                                  N/A
LINK                                                                                 N/A
Pendulum Scale                                                                       N/A
Card Description                       Activate this card by targeting 1 monster in y...
Card/Attribute/Type Support                                                           {}
Direct Archetype & Se

In [41]:
# This block of code is reserved for correcting any erroneous cards that were found
card_database['Card Type'].loc['Forbidden Dark Contract with the Swamp King'] = 'Spell'
card_database['Competitive Status (TCG Advanced)'].loc['Forbidden Dark Contract with the Swamp King'] = 'Unlimited'
card_database['Card Type'].loc["Ma'at"] = 'Monster'
card_database['Competitive Status (TCG Advanced)'].loc["Ma'at"] = 'Unlimited'
#card_database['Card Type'].loc['Call of the Haunted'] = 'Trap'
#card_database['Spell/Trap Property'].loc['Call of the Haunted'] = 'Continuous'
#card_database['Card Description'].loc['Call of the Haunted'] = 'Activate this card by targeting 1 monster in your GY; Special Summon that target in Attack Position. When this card leaves the field, destroy that monster. When that monster is destroyed, destroy this card.'
#card_database['Competitive Status (TCG Advanced)'].loc['Call of the Haunted'] = 'Unlimited'

In [42]:
card_database[card_database['Competitive Status (TCG Advanced)'] == 'Legal']

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Photon Token,Monster,N/A,LIGHT,Thunder / Token,4,2000,0,N/A,N/A,"This card can be used as a ""Photon Token"". *If...",{},{Photon},{},Legal,https://yugioh.fandom.com/wiki/Photon_Token
Kuriboh Token,Monster,N/A,DARK,Fiend / Token,1,300,200,N/A,N/A,"This card can be used as a ""Kuriboh Token"". *I...",{},{Kuriboh},{Signature move},Legal,https://yugioh.fandom.com/wiki/Kuriboh_Token
Synthetic Seraphim Token,Monster,N/A,LIGHT,Fairy / Token,1,300,300,N/A,N/A,"This card can be used as a ""Synthetic Seraphim...",{},{Counter Fairy},{},Legal,https://yugioh.fandom.com/wiki/Synthetic_Serap...
Phantasm Token,Monster,N/A,DARK,Fiend / Token,1,1000,1000,N/A,N/A,"This card can be used as a ""Phantasm Token"". *...",{},{Phantasm},{Sacred Beast},Legal,https://yugioh.fandom.com/wiki/Phantasm_Token
Hippo Token,Monster,N/A,EARTH,Beast / Token,1,0,0,N/A,N/A,"This card can be used as a ""Hippo Token"". *If ...",{},{},{},Legal,https://yugioh.fandom.com/wiki/Hippo_Token
Jurraegg Token,Monster,N/A,EARTH,Dinosaur / Token,1,0,0,N/A,N/A,"This card can be used as a ""Juraegg Token"". *I...",{},{},{},Legal,https://yugioh.fandom.com/wiki/Jurraegg_Token
Ancient Gear Token,Monster,N/A,EARTH,Machine / Token,1,0,0,N/A,N/A,"This card can be used as an ""Ancient Gear Toke...",{},{Ancient Gear},{},Legal,https://yugioh.fandom.com/wiki/Ancient_Gear_Token
Dragon Lord Token,Monster,N/A,LIGHT,Dragon / Token,1,0,0,N/A,N/A,"This card can be used as a ""Dragon Lord Token""...",{},{},{Felgrand},Legal,https://yugioh.fandom.com/wiki/Dragon_Lord_Token
Phantasmal Martyr Token,Monster,N/A,DARK,Fiend / Token,1,0,0,N/A,N/A,"This card can be used as a ""Phantasmal Martyr ...",{},{Phantasm},{Sacred Beast},Legal,https://yugioh.fandom.com/wiki/Phantasmal_Mart...


In [43]:
card_database = card_database[(card_database['Card Type'] == 'Monster') | (card_database['Card Type'] == 'Spell') | (card_database['Card Type'] == 'Trap')]
card_database = card_database[card_database['Competitive Status (TCG Advanced)'] != 'Legal']
card_database

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Divine Grace - Northwemko,Monster,N/A,LIGHT,Spellcaster / Ritual / Effect,7,2700,1200,N/A,N/A,This card can only be Ritual Summoned with the...,{},{},{Djinn of Rituals},Unlimited,https://yugioh.fandom.com/wiki/Divine_Grace_-_...
Double Summon,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,You can conduct 2 Normal Summons/Sets this tur...,{},{},{},Unlimited,https://yugioh.fandom.com/wiki/Double_Summon
Zoodiac Gathering,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"If a ""Zoodiac"" Xyz Monster you control would d...",{Xyz Monster},{Zoodiac},{},Unlimited,https://yugioh.fandom.com/wiki/Zoodiac_Gathering
The Legendary Exodia Incarnate,Monster,N/A,DARK,Spellcaster / Effect,10,?,0,N/A,N/A,Cannot be Normal Summoned/Set. Must be Special...,{},"{Forbidden One, Exodia}",{},Unlimited,https://yugioh.fandom.com/wiki/The_Legendary_E...
Dark Designator,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Declare 1 Monster Card name. If the declared c...,{},{Designator},{},Unlimited,https://yugioh.fandom.com/wiki/Dark_Designator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iron Hans,Monster,N/A,EARTH,Warrior / Effect,4,1200,800,N/A,N/A,If this card is Summoned: You can Special Summ...,"{Iron Knight, Golden Castle of Stromberg}",{Fairy Tale},{},Unlimited,https://yugioh.fandom.com/wiki/Iron_Hans
Frightfur March,Trap,Counter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"When an opponent's Spell/Trap Card, or monster...",{Fusion Monster},{Frightfur},{},Unlimited,https://yugioh.fandom.com/wiki/Frightfur_March
Shapesister,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Special Summon this card as a Normal Monster (...,{},{Trap Monster},{},Unlimited,https://yugioh.fandom.com/wiki/Shapesister


In [44]:
card_database.loc['Maxx "C"']

Card Type                                                                        Monster
Spell/Trap Property                                                                  N/A
Attribute                                                                          EARTH
Types                                                                    Insect / Effect
Level/Rank                                                                             2
ATK                                                                                  500
DEF                                                                                  200
LINK                                                                                 N/A
Pendulum Scale                                                                       N/A
Card Description                       During either player's turn: You can send this...
Card/Attribute/Type Support                                                           {}
Direct Archetype & Se

In [45]:
card_database['Competitive Status (TCG Advanced)'].unique()

array(['Unlimited', 'Limited', 'Forbidden', 'Semi-Limited', 'Illegal',
       'Not', 'Not yet released'], dtype=object)

In [47]:
card_database[card_database['Competitive Status (TCG Advanced)'] == 'Not']

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Toon Terror,Trap,Counter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"When a Spell/Trap Card, or monster effect, is ...","{Toon monster, Toon World}",{Toon},{},Not,https://yugioh.fandom.com/wiki/Toon_Terror
"Dark Armed, the Dragon of Annihilation",Monster,N/A,DARK,Dragon / Xyz / Effect,7,2800,1000,N/A,N/A,"2+ Level 7 monsters. Once per turn, if you hav...","{Dragon, DARK}",{},"{Armed Dragon, Dark counterpart}",Not,"https://yugioh.fandom.com/wiki/Dark_Armed,_the..."


In [49]:
card_database['Competitive Status (TCG Advanced)'].loc['Toon Terror'] = 'Not yet released'
card_database['Competitive Status (TCG Advanced)'].loc['Dark Armed, the Dragon of Annihilation'] = 'Not yet released'

In [50]:
card_database[card_database['Competitive Status (TCG Advanced)'] == 'Not yet released']

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Toon Terror,Trap,Counter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"When a Spell/Trap Card, or monster effect, is ...","{Toon monster, Toon World}",{Toon},{},Not yet released,https://yugioh.fandom.com/wiki/Toon_Terror
Grand Spiritual Art - Ichirin,Spell,Field,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"Once per turn, if you control a Spellcaster mo...",{},{Spiritual Art},{Charmer},Not yet released,https://yugioh.fandom.com/wiki/Grand_Spiritual...
"Dark Armed, the Dragon of Annihilation",Monster,N/A,DARK,Dragon / Xyz / Effect,7,2800,1000,N/A,N/A,"2+ Level 7 monsters. Once per turn, if you hav...","{Dragon, DARK}",{},"{Armed Dragon, Dark counterpart}",Not yet released,"https://yugioh.fandom.com/wiki/Dark_Armed,_the..."
In Touch with the Cosmos,Spell,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,You can send 1 monster you control that is own...,"{Machine, Jinzo}",{},{Jinzo},Not yet released,https://yugioh.fandom.com/wiki/In_Touch_with_t...
Dragunity Knight - Areadbhair,Monster,N/A,WIND,Dragon / Synchro / Effect,10,3300,3200,N/A,N/A,"1 ""Dragunity"" Tuner + 1+ non-Tuner monsters. W...",{},{Dragunity},{},Not yet released,https://yugioh.fandom.com/wiki/Dragunity_Knigh...
Awakened-Possessed - Great Inari Fire,Monster,N/A,FIRE,Pyro / Effect,5,2000,200,N/A,N/A,You can Special Summon this card (from your ha...,"{FIRE, Spellcaster}","{Possessed, Spiritual Fire Art, Awakened-Posse...",{},Not yet released,https://yugioh.fandom.com/wiki/Awakened-Posses...
Awakened-Possessed - Archfiend Reaper of Nefariousness,Monster,N/A,EARTH,Beast / Effect,5,2000,200,N/A,N/A,You can Special Summon this card (from your ha...,"{EARTH, Spellcaster}","{Spiritual Earth Art, Possessed, Awakened-Poss...",{},Not yet released,https://yugioh.fandom.com/wiki/Awakened-Posses...


In [53]:
card_database

,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
Card Name,,,,,,,,,,,,,,,
Divine Grace - Northwemko,Monster,N/A,LIGHT,Spellcaster / Ritual / Effect,7,2700,1200,N/A,N/A,This card can only be Ritual Summoned with the...,{},{},{Djinn of Rituals},Unlimited,https://yugioh.fandom.com/wiki/Divine_Grace_-_...
Double Summon,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,You can conduct 2 Normal Summons/Sets this tur...,{},{},{},Unlimited,https://yugioh.fandom.com/wiki/Double_Summon
Zoodiac Gathering,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"If a ""Zoodiac"" Xyz Monster you control would d...",{Xyz Monster},{Zoodiac},{},Unlimited,https://yugioh.fandom.com/wiki/Zoodiac_Gathering
The Legendary Exodia Incarnate,Monster,N/A,DARK,Spellcaster / Effect,10,?,0,N/A,N/A,Cannot be Normal Summoned/Set. Must be Special...,{},"{Forbidden One, Exodia}",{},Unlimited,https://yugioh.fandom.com/wiki/The_Legendary_E...
Dark Designator,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Declare 1 Monster Card name. If the declared c...,{},{Designator},{},Unlimited,https://yugioh.fandom.com/wiki/Dark_Designator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iron Hans,Monster,N/A,EARTH,Warrior / Effect,4,1200,800,N/A,N/A,If this card is Summoned: You can Special Summ...,"{Iron Knight, Golden Castle of Stromberg}",{Fairy Tale},{},Unlimited,https://yugioh.fandom.com/wiki/Iron_Hans
Frightfur March,Trap,Counter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"When an opponent's Spell/Trap Card, or monster...",{Fusion Monster},{Frightfur},{},Unlimited,https://yugioh.fandom.com/wiki/Frightfur_March
Shapesister,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Special Summon this card as a Normal Monster (...,{},{Trap Monster},{},Unlimited,https://yugioh.fandom.com/wiki/Shapesister


In [51]:
os.getcwd() 

'C:\\Users\\jorda\\OneDrive\\Desktop\\Personal Projects\\Yugioh Web Scraper'

In [54]:
card_database.to_csv('yugioh/Data/Yugioh Card Database (Backup).csv')

In [56]:
card_database.to_csv('yugioh/Data/Yugioh Card Database.csv')

In [57]:
import yugioh

In [63]:
card_database = yugioh.get_card_database()
card_database

,Card Name,Card Type,Spell/Trap Property,Attribute,Types,Level/Rank,ATK,DEF,LINK,Pendulum Scale,Card Description,Card/Attribute/Type Support,Direct Archetype & Series Support,Indirect Archetype & Series Support,Competitive Status (TCG Advanced),Reference
0,Divine Grace - Northwemko,Monster,N/A,LIGHT,Spellcaster / Ritual / Effect,7,2700,1200,N/A,N/A,This card can only be Ritual Summoned with the...,set(),set(),{'Djinn of Rituals'},Unlimited,https://yugioh.fandom.com/wiki/Divine_Grace_-_...
1,Double Summon,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,You can conduct 2 Normal Summons/Sets this tur...,set(),set(),set(),Unlimited,https://yugioh.fandom.com/wiki/Double_Summon
2,Zoodiac Gathering,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"If a ""Zoodiac"" Xyz Monster you control would d...",{'Xyz Monster'},{'Zoodiac'},set(),Unlimited,https://yugioh.fandom.com/wiki/Zoodiac_Gathering
3,The Legendary Exodia Incarnate,Monster,N/A,DARK,Spellcaster / Effect,10,?,0,N/A,N/A,Cannot be Normal Summoned/Set. Must be Special...,set(),"{'Forbidden One', 'Exodia'}",set(),Unlimited,https://yugioh.fandom.com/wiki/The_Legendary_E...
4,Dark Designator,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Declare 1 Monster Card name. If the declared c...,set(),{'Designator'},set(),Unlimited,https://yugioh.fandom.com/wiki/Dark_Designator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10386,Iron Hans,Monster,N/A,EARTH,Warrior / Effect,4,1200,800,N/A,N/A,If this card is Summoned: You can Special Summ...,"{'Iron Knight', 'Golden Castle of Stromberg'}",{'Fairy Tale'},set(),Unlimited,https://yugioh.fandom.com/wiki/Iron_Hans
10387,Frightfur March,Trap,Counter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"When an opponent's Spell/Trap Card, or monster...",{'Fusion Monster'},{'Frightfur'},set(),Unlimited,https://yugioh.fandom.com/wiki/Frightfur_March
10388,Shapesister,Trap,Continuous,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Special Summon this card as a Normal Monster (...,set(),{'Trap Monster'},set(),Unlimited,https://yugioh.fandom.com/wiki/Shapesister
10389,Gotterdammerung,Spell,Normal,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"Target 1 ""Aesir"" monster you control; give con...",set(),set(),set(),Unlimited,https://yugioh.fandom.com/wiki/Gotterdammerung


In [62]:
yugioh.save_card_database(card_database)

Save successful


In [117]:
card_database.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7375 entries, 0 to 7374
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Card Name                            7375 non-null   object
 1   Card Type                            7375 non-null   object
 2   Spell/Trap Property                  7375 non-null   object
 3   Attribute                            7375 non-null   object
 4   Types                                7375 non-null   object
 5   Level/Rank                           7375 non-null   object
 6   ATK                                  7375 non-null   object
 7   DEF                                  7375 non-null   object
 8   LINK                                 7375 non-null   object
 9   Pendulum Scale                       7375 non-null   object
 10  Card Description                     7375 non-null   object
 11  Card/Attribute/Type Support          7375 n